# Stage 3 - Entity Mathcing 

In [1]:
from collections import Counter
import os

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

import py_entitymatching as em

IS_DEVELOPING = True

data_dir = './dataset/structured_data/'
A_filename = 'forbes_sample.csv' if IS_DEVELOPING else 'forbes_all.csv'
B_filename = 'nasdaq_sample.csv' if IS_DEVELOPING else 'nasdaq.csv'
blocked_filename = 'blocked_sample.csv' if IS_DEVELOPING else 'blocked.csv'

all_fields = ['Name', 'Industry', 'MarketValue']

## Blocking

In [2]:
A = pd.read_csv(data_dir + A_filename)
em.set_key(A, 'id')
B = pd.read_csv(data_dir + B_filename)
em.set_key(B, 'id')

True

Using overlap blocker.

In [3]:
ob = em.OverlapBlocker()

Rule out common words.

In [4]:
ob.stop_words.extend(w.lower() for w in [
    'of',
    'property', 'holdings', 'holding', 'inc', 'bancorp', 'Bancorporation', 'plc',
    'Corporation', 'Corp', 'Group', 'Company',
    'Advanced',
    'Insurance', 'Bank', 'Financial', 'Pharmaceuticals', 'Pharma', 'Therapeutics',
    'Systems', 'Technologies', 'Technology', 'Express', 'Air', 'Securities', 'Security'
    'Mining', 'Energy', 'Electric', 'Gas', 'Oil', 'Power'])
ob.stop_words = list(set(ob.stop_words))
print(*ob.stop_words, sep=', ')

its, it, gas, electric, has, will, as, holdings, to, from, technology, at, energy, of, and, pharma, by, power, corp, property, group, with, advanced, be, the, therapeutics, bank, company, was, systems, securitymining, that, are, on, in, securities, a, an, for, corporation, oil, holding, plc, financial, technologies, inc, insurance, is, air, pharmaceuticals, bancorp, were, bancorporation, express, he


In [5]:
## jz: not yet sure how to combine multiple occurrence of same companies, e.g. "Zions Bancorp" vs "Zions Bancorp."
# AA = ob.block_tables(A, A, 'Name', 'Name', word_level=True, overlap_size=1, rem_stop_words=True,
#                     l_output_attrs=all_fields, 
#                     r_output_attrs=all_fields,
#                     show_progress=False)
# print(len(AA))
# AA.head()

In [6]:
# Specify the tokenization to be 'word' level and set overlap_size to be 3.
blocked = ob.block_tables(A, B, 'Name', 'Name', word_level=True, overlap_size=1, rem_stop_words=True,
                    l_output_attrs=all_fields, 
                    r_output_attrs=all_fields,
                    show_progress=False)
print('# pairs:', len(blocked))
# Display first 5 tuple pairs in the candidate set.
blocked.head()

# pairs: 56


,_id,ltable_id,rtable_id,ltable_Name,ltable_Industry,ltable_MarketValue,rtable_Name,rtable_Industry,rtable_MarketValue
0,0,2,9,3M,Conglomerates,102200,3M Company,Medical/Dental Instruments,$111.73B
1,1,7,16,A. Schulman,Chemicals,798,"A. Schulman, Inc.",Major Chemicals,$992.75M
2,2,11,24,ABB,Conglomerates,48100,ABB Ltd,Electrical Products,$48.17B
3,3,12,25,Abbott Laboratories,Pharmaceuticals,64900,Abbott Laboratories,Major Pharmaceuticals,$78.93B
4,4,13,26,AbbVie,Pharmaceuticals,99400,AbbVie Inc.,Major Pharmaceuticals,$98.97B


In [7]:
# try out blocker over a single pair
# ob.block_tuples(A.ix[60], B.ix[0], l_overlap_attr='Name', r_overlap_attr='Name',
#                rem_stop_words=True, word_level=True, overlap_size=1)

In [8]:
## debug and improve the blocker
## This may take some trial and run...

Finally save the results from blocking step.

In [9]:
blocked.to_csv(data_dir + blocked_filename)